In [1]:
%load_ext autoreload
%autoreload 2
import clipper_manager as cl
import os
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.externals import joblib
import sys
import json


## Connect to EC2

In [2]:
import clipper_manager as cl
ec2_host = "ec2-52-53-148-130.us-west-1.compute.amazonaws.com"
user = "ubuntu"
key = os.path.expanduser("~/.ssh/aws_rsa")
clipper = cl.Cluster(ec2_host, user, key)

Checking if Docker running...
Found Docker running
Creating internal Docker network
Creating local model repository


## Start Clipper

In [3]:
clipper.start_clipper()

starting Clipper with default settings:
models = []
redis_port = 6379
use_lsh = false
window_size = -1
name = "clipper-demo"
input_type = "float"
num_update_workers = 1
num_predict_workers = 1
redis_ip = "redis-clipper"
cache_size = 49999
slo_micros = 20000
correction_policy = "logistic_regression"
input_length = 784
[batching]
sample_size = 1000
strategy = "aimd"

[ec2-52-53-148-130.us-west-1.compute.amazonaws.com] sudo: docker run -d --network=clipper_nw -p 6379:6379 --cpuset-cpus="0" --name redis-clipper redis:alpine
[ec2-52-53-148-130.us-west-1.compute.amazonaws.com] sudo: docker run -d --network=clipper_nw -p 1337:1337 --cpuset-cpus="1-4" --name clipper -v ~/conf.toml:/tmp/conf.toml dcrankshaw/clipper


In [14]:
print json.dumps(clipper.get_metrics(), indent=4)

{
    "meters": [
        {
            "rate": 7.766437969338588, 
            "name": "prediction_thruput", 
            "unit": "events per second"
        }, 
        {
            "rate": 0.0, 
            "name": "update_thruput", 
            "unit": "events per second"
        }, 
        {
            "rate": 1.0921552069045566, 
            "name": "bad_rf_model:model_thruput", 
            "unit": "events per second"
        }, 
        {
            "rate": 3.6405173563485222, 
            "name": "spark_svm:model_thruput", 
            "unit": "events per second"
        }, 
        {
            "rate": 7.523735869786946, 
            "name": "rbf_svm_model:model_thruput", 
            "unit": "events per second"
        }
    ], 
    "histograms": [
        {
            "std": 16.645157251303903, 
            "p99": 20212.0, 
            "name": "prediction_latency", 
            "min": 20142, 
            "max": 20212, 
            "p95": 20210.75, 
            "p50": 

## Start a serving workload

We go to a [different notebook](run_serving_workload.ipynb) so we can start querying the model from a separate process. 

# Train a Scikit-Learn model

In [5]:
def load_digits(digits_location, digits_filename = "train.data", norm=True):
    digits_path = digits_location + "/" + digits_filename
    print("Source file: %s" % digits_path)
    df = pd.read_csv(digits_path, sep=",", header=None)
    data = df.values
    print("Number of image files: %d" % len(data))
    y = data[:,0]
    X = data[:,1:]
    Z = X
    if norm:
        mu = np.mean(X,0)
        sigma = np.var(X,0)
        Z = (X - mu) / np.array([np.sqrt(z) if z > 0 else 1. for z in sigma])
    return Z, y

def filter_data(data):
    cx, cy = data
    binary_x = []
    binary_y = []
    for i in range(len(cy)):
        if cy[i] == 3:
            binary_x.append(cx[i,:])
            binary_y.append(1.0)
        elif cy[i] == 6:
            binary_x.append(cx[i,:])
            binary_y.append(0.0)
    return np.array(binary_x), np.array(binary_y)
            
train_x, train_y = filter_data(load_digits(os.path.expanduser("~/model-serving/data/mnist_data")))
test_x, test_y = filter_data(load_digits(os.path.expanduser("~/model-serving/data/mnist_data"), digits_filename="test.data"))

Source file: /Users/crankshaw/model-serving/data/mnist_data/train.data
Number of image files: 60000
Source file: /Users/crankshaw/model-serving/data/mnist_data/test.data
Number of image files: 10000


## Train a bad model

In [6]:
rf_model = RFC(n_estimators=2, max_depth=1)
rf_model.fit(train_x, train_y)
rf_model.score(test_x, test_y)

0.83523908523908519

## Deploy A Bad Model

In [7]:
clipper.add_sklearn_model("bad_rf_model", rf_model)

/tmp/clipper-models/bad_rf_model/1
[ec2-52-53-148-130.us-west-1.compute.amazonaws.com] sudo: docker run -d --network=clipper_nw --name bad_rf_model_v1_r0 -v /tmp/clipper-models/bad_rf_model/1/bad_rf_model:/model:ro dcrankshaw/clipper-sklearn-mw


## Train a Spark Model

Now let's go train a model using Spark in a Databricks Cloud [Notebook](https://amplab-berkeley-research.cloud.databricks.com/#notebook/46987).

## Deploy a Spark Model from S3

In [9]:
clipper.add_pyspark_model("spark_svm", "s3://clipperdbdemo/spark_svm_model/svm_predict_3")

/tmp/clipper-models/spark_svm/1
[ec2-52-53-148-130.us-west-1.compute.amazonaws.com] run: aws s3 cp s3://clipperdbdemo/spark_svm_model/svm_predict_3 /tmp/clipper-models/spark_svm/1/svm_predict_3 --recursive
[ec2-52-53-148-130.us-west-1.compute.amazonaws.com] out: download: s3://clipperdbdemo/spark_svm_model/svm_predict_3/data/_SUCCESS to svm_predict_3/data/_SUCCESS
[ec2-52-53-148-130.us-west-1.compute.amazonaws.com] out: Completed 1 of 6 part(s) with 5 file(s) remaining
[ec2-52-53-148-130.us-west-1.compute.amazonaws.com] out: download: s3://clipperdbdemo/spark_svm_model/svm_predict_3/metadata/part-00000 to svm_predict_3/metadata/part-00000
[ec2-52-53-148-130.us-west-1.compute.amazonaws.com] out: Completed 2 of 6 part(s) with 4 file(s) remaining
[ec2-52-53-148-130.us-west-1.compute.amazonaws.com] out: download: s3://clipperdbdemo/spark_svm_model/svm_predict_3/data/part-r-00000-617453e6-2671-4674-a07d-47d0fb49871e.gz.parquet to svm_predict_3/data/part-r-00000-617453e6-2671-4674-a07d-47d0f

## Train an SVM with RBF Kernel

In [11]:
svm_model = svm.SVC()
svm_model.fit(train_x, train_y)
svm_model.score(test_x, test_y)

0.99480249480249483

In [12]:
clipper.add_sklearn_model("rbf_svm_model", svm_model)

/tmp/clipper-models/rbf_svm_model/1
[ec2-52-53-148-130.us-west-1.compute.amazonaws.com] sudo: docker run -d --network=clipper_nw --name rbf_svm_model_v1_r0 -v /tmp/clipper-models/rbf_svm_model/1/rbf_svm_model:/model:ro dcrankshaw/clipper-sklearn-mw


In [15]:
print json.dumps(clipper.get_correction_model(0), indent=4)

{
    "linear_model": {
        "label": [
            1, 
            0
        ], 
        "bias": -1.0, 
        "nr_class": 2, 
        "w": [
            2.1981558952222655
        ], 
        "nr_feature": 1
    }, 
    "anytime_estimators": [
        0.44
    ], 
    "offline_model_order": [
        "rbf_svm_model", 
        "bad_rf_model", 
        "spark_svm"
    ]
}


## Send some corrections

We go to a [different notebook](send_updates.ipynb) to send more training data to Clipper. 

## Stop Clipper

In [ ]:
clipper.stop_all()

In [ ]:
lm_model = lm.LogisticRegression()
lm_model.fit(train_x, train_y)
lm_model.score(test_x, test_y)

In [ ]:
clipper.add_replicas("bad_rf_model", 1, num_replicas=3)